# VacationPy
----

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

import json
import requests

# Import API key
from api_keys import g_key
from api_keys import weather_api_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
# read the csv into a dataframe
cities_df = pd.read_csv('./citiesA.csv')
# drop any na rows
cities_df.dropna()
# Check the dataframe by examining just the head
cities_df.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Tuktoyaktuk,90,CA,1584112497,77,69.45,-133.04,3.20,19.46
1,1,Tazmalt,100,DZ,1584112439,68,36.39,4.40,68.00,10.29
2,2,Naze,90,JP,1584112675,100,28.37,129.48,62.60,1.12
3,3,Burgeo,31,CA,1584112910,66,47.62,-57.62,23.63,7.65
4,4,Dunedin,30,NZ,1584112688,86,-45.87,170.50,52.00,4.88


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
# Configure gmaps
gmaps.configure(api_key=g_key)
# Store latitude and longitude in locations
locations = cities_df[["Lat", "Lng"]]
# Store humidity information in humidity
humidity = cities_df["Humidity"].astype(float)

In [6]:
# Customize the size of the figure
figure_layout = {
    'width': '900px',
    'height': '900px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)
coordinates = cities_df[["Lat", "Lng"]]

In [7]:
# Plot Heatmap
fig = gmaps.figure(layout=figure_layout,center=(0.0, 0.0), zoom_level=1.8)

# Create heatmap layer
heat_layer = gmaps.heatmap_layer(coordinates, weights=humidity, 
                                 dissipating=False, max_intensity=max(humidity),
                                 point_radius=1)

# Add heatmap layer
fig.add_layer(gmaps.heatmap_layer(coordinates, weights=humidity))

# Display figure
fig


Figure(layout=FigureLayout(border='1px solid black', height='900px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
# A max temperature lower than 80 degrees but higher than 60.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
hotel_df =   cities_df[(cities_df['Max Temp'] >= 60) 
          & (cities_df['Max Temp'] < 80) 
          & (cities_df['Wind Speed'] < 10)
          & (cities_df['Cloudiness'] == 0)].reset_index()

# set all the values to '' for hotel name
hotel_df.insert(loc=1, column='Hotel Name', value=['' for i in range(hotel_df.shape[0])])
#hotel_df

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Loop through each row finding the nearest hotel by lat,lng coordinates
# and populate the 'Hotel Name' column with the closest hotel name
# The hotel names are also printed to the screen

for index, row in hotel_df.iterrows():
    # put the lat,lng values into string location
    # so it can help build the query
    location = f"{row['Lat']}, {row['Lng']}"
    # set up parameters
    params = {
        "location": location,  # city coords,
        "rankby": "distance",
        "type": "lodging",
        "key": g_key,
    }
    
    # print(params)   # for debugging    
    # make the request
    response = requests.get(base_url, params=params).json()
   
    # extract results
    results = response['results']
       
    try:
        print(f"The closest hotel to the city of {row['City']} is the {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = {results[0]['name']}      
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------------------------------------------------------")
    
#hotel_df['Hotel Name']
#hotel_df

The closest hotel to the city of Hirara is the ゲストハウスfamilia.
------------------------------------------------------------
The closest hotel to the city of Henties Bay is the Eagle Accommodation.
------------------------------------------------------------
The closest hotel to the city of Hermanus is the Hermanus Bed and Breakfast.
------------------------------------------------------------
The closest hotel to the city of Mount Isa is the Aboriginal Hostels Ltd..
------------------------------------------------------------
The closest hotel to the city of Batticaloa is the Doctors' Official Residence.
------------------------------------------------------------
The closest hotel to the city of Bāndarban is the Hotel Sangu.
------------------------------------------------------------
The closest hotel to the city of San Ciro de Acosta is the Hotel Maria Isabel.
------------------------------------------------------------
The closest hotel to the city of Lázaro Cárdenas is the Hoteles 

In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]
#locations  # for debugging
    

In [13]:
# Add marker layer ontop of heat map
# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations, info_box_content=hotel_info, hover_text=hotel_df['Hotel Name'])

# Add the layer to the map
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(border='1px solid black', height='900px', margin='0 auto 0 auto', padding='1px', wi…

### End of notebook